# Keras Multi-layer Perceptron

Nothing crazy.

### Imports (from the UNIX Server)

In [168]:
import os
import numpy as np
import tensorflow as tf
from keras import backend as K


class GenericModel:
    """
    Generic Tensorflow Model Class

    Each subclass of this class needs to define the data structure of its weights
    (which should be respected accross methods) and implement the functions below.

    """

    def get_model(self):
        raise NotImplementedError("Subclasses should implement this!")

    def load_weights(self):
        raise NotImplementedError("Subclasses should implement this!")

    def get_weights(self):
        raise NotImplementedError("Subclasses should implement this!")

    def sum_weights(self, weights1, weights2):
        raise NotImplementedError("Subclasses should implement this!")

    def scale_weights(self, weights, factor):
        raise NotImplementedError("Subclasses should implement this!")

    def inverse_scale_weights(self, weights, factor):
        raise NotImplementedError("Subclasses should implement this!")

class GenericKerasModel(GenericModel):
    def set_weights(self, new_weights):
        self.model.set_weights(new_weights)

    def get_weights(self):
        return self.model.get_weights()

    def get_initial_weights(self):
        model = self.build_model()
        return model.get_weights()

    def sum_weights(self, weights1, weights2):
        new_weights = []
        for w1, w2 in zip(weights1, weights2):
            new_weights.append(w1 + w2)
        return new_weights

    def scale_weights(self, weights, factor):
        new_weights = []
        for w in weights:
            new_weights.append(w * factor)
        return new_weights

    def inverse_scale_weights(self, weights, factor):
        new_weights = []
        for w in weights:
            new_weights.append(w / factor)
        return new_weights


In [169]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers


class KerasPerceptron(GenericKerasModel):
    def __init__(self, is_training=False):
        self.n_input = 784
        self.n_hidden1 = 200
        self.n_hidden2 = 200
        self.n_classes = 10
        self.is_training = is_training
        self.model = self.build_model()
        if is_training:
            self.compile_model()

    def build_model(self):
        model = Sequential()
        model.add(Dense(self.n_hidden1, input_shape=(self.n_input,), activation='relu', kernel_initializer=keras.initializers.glorot_uniform()))
        model.add(Dense(self.n_hidden2, activation='relu', kernel_initializer=keras.initializers.glorot_uniform()))
        model.add(Dense(self.n_classes, activation='softmax', kernel_initializer=keras.initializers.glorot_uniform()))
        # model.summary()
        return model

    def compile_model(self):
        sgd = optimizers.SGD(lr=0.001)
        self.model.compile(
            optimizer=sgd,
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )

In [170]:
"""
Custom saving/loading for Keras.
Based on https://github.com/keras-team/keras/blob/master/keras/engine/saving.py.
"""

import json

from keras import optimizers

from keras.models import model_from_json


def model_from_serialized(serialized_model):
    uncompiled_model = model_from_json(serialized_model['architecture'])
    return _load_optimizer(uncompiled_model, serialized_model['optimizer'])

def get_optimizer(model):
    def get_json_type(obj):
        """Serialize any object to a JSON-serializable structure.
        # Arguments
            obj: the object to serialize
        # Returns
            JSON-serializable structure representing `obj`.
        # Raises
            TypeError: if `obj` cannot be serialized.
        """
        # if obj is a serializable Keras class instance
        # e.g. optimizer, layer
        if hasattr(obj, 'get_config'):
            return {'class_name': obj.__class__.__name__,
                    'config': obj.get_config()}

        # if obj is any numpy type
        if type(obj).__module__ == np.__name__:
            if isinstance(obj, np.ndarray):
                return {'type': type(obj),
                        'value': obj.tolist()}
            else:
                return obj.item()

        # misc functions (e.g. loss function)
        if callable(obj):
            return obj.__name__

        # if obj is a python 'type'
        if type(obj).__name__ == type.__name__:
            return obj.__name__

        raise TypeError('Not JSON Serializable:', obj)

    if model.optimizer:
        metadata = {}
        if isinstance(model.optimizer, optimizers.TFOptimizer):
            warnings.warn(
                'TensorFlow optimizers do not '
                'make it possible to access '
                'optimizer attributes or optimizer state '
                'after instantiation. '
                'As a result, we cannot save the optimizer '
                'as part of the model save file.'
                'You will have to compile your model again '
                'after loading it. '
                'Prefer using a Keras optimizer instead '
                '(see keras.io/optimizers).')
        else:
            metadata['training_config'] = json.dumps({
                'optimizer_config': {
                    'class_name': model.optimizer.__class__.__name__,
                    'config': model.optimizer.get_config()
                },
                'loss': model.loss,
                'metrics': model.metrics,
                'sample_weight_mode': model.sample_weight_mode,
                'loss_weights': model.loss_weights,
            }, default=get_json_type)

    return metadata

def _load_optimizer(uncompiled_model, optimizer_metadata):
    custom_objects = {}

    def convert_custom_objects(obj):
        """Handles custom object lookup.
        # Arguments
            obj: object, dict, or list.
        # Returns
            The same structure, where occurrences
                of a custom object name have been replaced
                with the custom object.
        """
        if isinstance(obj, list):
            deserialized = []
            for value in obj:
                deserialized.append(convert_custom_objects(value))
            return deserialized
        if isinstance(obj, dict):
            deserialized = {}
            for key, value in obj.items():
                deserialized[key] = convert_custom_objects(value)
            return deserialized
        if obj in custom_objects:
            return custom_objects[obj]
        return obj

    # instantiate optimizer
    training_config = optimizer_metadata.get('training_config')
    if training_config is None:
        warnings.warn('No training configuration found in save file: '
                      'the model was *not* compiled. '
                      'Compile it manually.')
        return uncompiled_model
    training_config = json.loads(training_config)
    optimizer_config = training_config['optimizer_config']
    optimizer = optimizers.deserialize(optimizer_config,
                                       custom_objects=custom_objects)

    # Recover loss functions and metrics.
    loss = convert_custom_objects(training_config['loss'])
    metrics = convert_custom_objects(training_config['metrics'])
    sample_weight_mode = training_config['sample_weight_mode']
    loss_weights = training_config['loss_weights']

    # Compile model.
    uncompiled_model.compile(optimizer=optimizer,
                             loss=loss,
                             metrics=metrics,
                             loss_weights=loss_weights,
                             sample_weight_mode=sample_weight_mode)

    model = uncompiled_model
    return model


### Setting up model

In [87]:
m = KerasPerceptron(is_training=True)
m.model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_69 (Dense)             (None, 200)               157000    
_________________________________________________________________
dense_70 (Dense)             (None, 200)               40200     
_________________________________________________________________
dense_71 (Dense)             (None, 10)                2010      
Total params: 199,210
Trainable params: 199,210
Non-trainable params: 0
_________________________________________________________________


### Data loading

In [88]:
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train.resize((x_train.shape[0], 28 * 28))
x_test.resize((x_test.shape[0], 28 * 28))

### Evaluation (Initial Weights)

#### Preview of weights

In [89]:
m.get_weights()

[array([[-0.02772636, -0.04897893, -0.02742239, ..., -0.04553548,
         -0.04263868, -0.00225566],
        [-0.04189289, -0.00273237, -0.07241117, ...,  0.03103768,
          0.00918383, -0.04232153],
        [ 0.00949974, -0.05841019,  0.01852147, ..., -0.0653021 ,
         -0.00260213, -0.05574984],
        ...,
        [-0.03398181,  0.05027232,  0.04795089, ..., -0.01953086,
          0.03278097,  0.0472555 ],
        [-0.01158489,  0.01730023, -0.03912467, ..., -0.07445896,
         -0.02684168, -0.07745838],
        [-0.04570943, -0.04665513, -0.07514221, ...,  0.01580391,
          0.03125388,  0.01501066]], dtype=float32),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

#### Evaluation

In [90]:
results = m.model.evaluate(x=x_test, y=y_test)
print("\nEvaluation loss is {0} and accuracy is {1}".format(*results))

10000/10000 [==============================] - 0s 46us/step

Evaluation loss is 14.462377227783204 and accuracy is 0.0853


### Training the model

In [91]:
m.model.fit(x=x_train, y=y_train, batch_size=128, epochs=15)

Epoch 1/15
60000/60000 [==============================] - 2s 37us/step - loss: 8.1086 - acc: 0.4840
Epoch 2/15
60000/60000 [==============================] - 2s 32us/step - loss: 5.7738 - acc: 0.6318
Epoch 3/15
60000/60000 [==============================] - 2s 38us/step - loss: 4.5407 - acc: 0.7065
Epoch 4/15
60000/60000 [==============================] - 2s 35us/step - loss: 4.2289 - acc: 0.7282
Epoch 5/15
60000/60000 [==============================] - 2s 26us/step - loss: 4.0833 - acc: 0.7382
Epoch 6/15
60000/60000 [==============================] - 2s 26us/step - loss: 3.3220 - acc: 0.7854
Epoch 7/15
60000/60000 [==============================] - 2s 27us/step - loss: 2.1923 - acc: 0.8559
Epoch 8/15
60000/60000 [==============================] - 2s 28us/step - loss: 2.1216 - acc: 0.8606
Epoch 9/15
60000/60000 [==============================] - 2s 28us/step - loss: 2.0675 - acc: 0.8648
Epoch 10/15
60000/60000 [==============================] - 2s 28us/step - loss: 2.0189 - acc: 0.8683

### Evaluation (Trained Weights)

In [160]:
results = m.model.evaluate(x=x_test, y=y_test)
print("\nEvaluation loss is {0} and accuracy is {1}".format(*results))

10000/10000 [==============================] - 1s 61us/step

Evaluation loss is 0.6063993990784547 and accuracy is 0.9518


### Saving the model

Ordered from best to worst.

#### 1. Using the model + optimizer + weights .h5

In [166]:
m.model.model.save('saved_mlp_model_with_w.h5') # NOTE THE model.model! IT'S IMPORTANT FOR WEIGHT ACCURACY!

In [164]:
# For some reason the model is worse than before saving. Could it be quantization? 
m2 = keras.models.load_model('saved_mlp_model_with_w.h5') 
results = m2.model.evaluate(x=x_test, y=y_test)
print("\nEvaluation loss is {0} and accuracy is {1}".format(*results))

10000/10000 [==============================] - 1s 61us/step

Evaluation loss is 0.6063993990784547 and accuracy is 0.9518


In [165]:
# This model can continue training!
m2.model.fit(x=x_train, y=y_train, batch_size=128, epochs=5)
results = m2.model.evaluate(x=x_test, y=y_test)
print("\nEvaluation loss is {0} and accuracy is {1}".format(*results))

Epoch 1/5
60000/60000 [==============================] - 2s 40us/step - loss: 0.4097 - acc: 0.9679
Epoch 2/5
60000/60000 [==============================] - 2s 32us/step - loss: 0.3667 - acc: 0.9721
Epoch 3/5
60000/60000 [==============================] - 2s 33us/step - loss: 0.3282 - acc: 0.9743
Epoch 4/5
60000/60000 [==============================] - 2s 33us/step - loss: 0.3020 - acc: 0.9769
Epoch 5/5
10000/10000 [==============================] - 0s 37us/step

Evaluation loss is 0.47373921380256034 and accuracy is 0.9616


#### 2. Using a hybrid (JSON for architecture and optimizer + .h5 for weights)

##### Saving

In [142]:
def serialize_model(keras_model):
    if not keras_model._is_compiled:
        raise Exception("Model needs to be compiled first.")
    return {
        'architecture': keras_model.to_json(),
        'optimizer': get_optimizer(keras_model)
    }

In [143]:
serialized_model = serialize_model(m.model)

In [144]:
serialized_model

{'architecture': '{"class_name": "Sequential", "config": {"name": "sequential_24", "layers": [{"class_name": "Dense", "config": {"name": "dense_69", "trainable": true, "batch_input_shape": [null, 784], "dtype": "float32", "units": 200, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "VarianceScaling", "config": {"scale": 1.0, "mode": "fan_avg", "distribution": "uniform", "seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}, {"class_name": "Dense", "config": {"name": "dense_70", "trainable": true, "units": 200, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "VarianceScaling", "config": {"scale": 1.0, "mode": "fan_avg", "distribution": "uniform", "seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, 

In [156]:
with open('model_w_serializer.json', 'w') as outfile:
    json.dump(serialized_model, outfile)

##### Loading

In [149]:
m5 = model_from_serialized(serialized_model)
assert m5._is_compiled, "Model is not compiled (UNEXPECTED)"

In [152]:
results = m5.model.evaluate(x=x_test, y=y_test)
print("\nEvaluation loss on INITIAL MODEL is {0} and accuracy is {1}".format(*results))

10000/10000 [==============================] - 0s 41us/step

Evaluation loss on INITIAL MODEL is 14.725766250610352 and accuracy is 0.0699


In [154]:
m5.load_weights('saved_mlp_weights.h5')
results = m5.model.evaluate(x=x_test, y=y_test)
print("\nEvaluation loss on PRETRAINED MODEL is {0} and accuracy is {1}".format(*results))

10000/10000 [==============================] - 0s 39us/step

Evaluation loss on PRETRAINED MODEL is 0.6063993990784547 and accuracy is 0.9518


In [155]:
# This model can continue training like we want it.
m5.model.fit(x=x_train, y=y_train, batch_size=128, epochs=5)
results = m5.model.evaluate(x=x_test, y=y_test)
print("\nEvaluation loss is {0} and accuracy is {1}".format(*results))

Epoch 1/5
60000/60000 [==============================] - 2s 41us/step - loss: 0.4059 - acc: 0.9679
Epoch 2/5
60000/60000 [==============================] - 2s 32us/step - loss: 0.3497 - acc: 0.9724
Epoch 3/5
60000/60000 [==============================] - 2s 38us/step - loss: 0.3222 - acc: 0.9747
Epoch 4/5
60000/60000 [==============================] - 2s 38us/step - loss: 0.2945 - acc: 0.9775
Epoch 5/5
10000/10000 [==============================] - 0s 36us/step

Evaluation loss is 0.4695930263816037 and accuracy is 0.9623


#### 3. Using the (only) weights .h5 

In [115]:
m.model.save_weights('saved_mlp_weights.h5')

In [116]:
m4 = KerasPerceptron(is_training=True)
m4.model.load_weights('saved_mlp_weights.h5')
results = m4.model.evaluate(x=x_test, y=y_test)
print("\nEvaluation loss is {0} and accuracy is {1}".format(*results))

10000/10000 [==============================] - 1s 61us/step

Evaluation loss is 0.6063993990784547 and accuracy is 0.9518


In [117]:
# This model can continue training, BUT only because we instantiated the KerasPerceptron class.
m4.model.fit(x=x_train, y=y_train, batch_size=128, epochs=5)
results = m4.model.evaluate(x=x_test, y=y_test)
print("\nEvaluation loss is {0} and accuracy is {1}".format(*results))

Epoch 1/5
60000/60000 [==============================] - 3s 50us/step - loss: 0.4066 - acc: 0.9679
Epoch 2/5
60000/60000 [==============================] - 2s 28us/step - loss: 0.3619 - acc: 0.9718
Epoch 3/5
60000/60000 [==============================] - 2s 30us/step - loss: 0.3180 - acc: 0.9748
Epoch 4/5
60000/60000 [==============================] - 2s 39us/step - loss: 0.3003 - acc: 0.9770
Epoch 5/5
10000/10000 [==============================] - 0s 40us/step

Evaluation loss is 0.46253317286027196 and accuracy is 0.9623


#### 4. Using the .npy

In [95]:
trained_weights = m.get_weights()
np.save('./perceptron_trained.npy', trained_weights)

In [108]:
# The accuracy is exactly what it was before :)
m3 = KerasPerceptron(is_training=True)
m3.set_weights(np.load('perceptron_trained.npy'))
results = m3.model.evaluate(x=x_test, y=y_test)
print("\nEvaluation loss is {0} and accuracy is {1}".format(*results))

10000/10000 [==============================] - 0s 48us/step

Evaluation loss is 0.6063993990784547 and accuracy is 0.9518


In [109]:
# This model can continue training, BUT only because we instantiated the KerasPerceptron class.
m3.model.fit(x=x_train, y=y_train, batch_size=128, epochs=5)
results = m3.model.evaluate(x=x_test, y=y_test)
print("\nEvaluation loss is {0} and accuracy is {1}".format(*results))

Epoch 1/5
60000/60000 [==============================] - 2s 38us/step - loss: 0.4039 - acc: 0.9683
Epoch 2/5
60000/60000 [==============================] - 2s 27us/step - loss: 0.3512 - acc: 0.9729
Epoch 3/5
60000/60000 [==============================] - 2s 31us/step - loss: 0.3233 - acc: 0.9758
Epoch 4/5
60000/60000 [==============================] - 2s 28us/step - loss: 0.3066 - acc: 0.9767
Epoch 5/5
10000/10000 [==============================] - 0s 34us/step

Evaluation loss is 0.4861069461776333 and accuracy is 0.9605


### Conclusion

When using Keras, the best way to encode the model (and optimizer & weights) and send it around for more training is to use the `model.model.save(filepath)` function. 

**Please notice the** `model.model` **because it's important! Without it the weights will change and the model won't be accurate.**

The exact lines of code to do this would be:

```python
# Serialize the model, optimizer, and weights
model = # The Keras model here
filepath = 'models/filename.h5'
model.model.save(filepath)
```

At this point, `serialized_model` will contain the model and optimizer while the file `weights/filename.h5` will contain the weights.

To send this information through an HTTP request we would do the following:

```python
url = 'https://servers.dataagora.com/start-dml'
metadata = {} # Some metadata
with open(weights_filepath, 'rb') as f:
    r = requests.post(url, files={'file': f}, data=metadata)
```

**NOTES**: It may be that the file we're sending is too big for either the client to send or the server to receive. In these cases, we can either stream or cut the file into chunks to send. Here are some resources to accomplish this: [one](https://stackoverflow.com/a/54857411), [two](https://stackoverflow.com/a/35784072), [three](http://docs.python-requests.org/en/latest/user/quickstart/#post-a-multipart-encoded-file_).

#### UPDATE

The conclusion above is accurate and probably necessary when using a *TensorFlow.js* trainer, but it's not sufficient. *TensorFlow.js* doesn't incorporate the optimizer information when deserializing the model, so the client needs to manually compile the model. (See the `KerasAndTFJS.ipynb` for more details.)